In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import math
import os
import csv

In [2]:
# 获取页面数据，将json object转化为python dict
def get_url(url):
    simulateBrowserData = {
        'Accept':'*/*',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Host': city + '.meituan.com', 
        'Referer':'https://' + city + '.meituan.com/meishi/', 
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.91 Safari/537.36'
    }
    response = requests.get(url, headers = simulateBrowserData)
    
    soup = BeautifulSoup(response.text)
    #for script in soup.find_all('script'):
    #    print(script.contents)
    all_scripts = soup.find_all('script')
    text = None
    for number, script in enumerate(all_scripts):
        if 'window._appState' in script.text:
            text = script.text
            #print(number, text)
    if text == None:
        print("服务器拒绝访问")
    else:
        data = json.loads(text.split("=",1)[1][:-1])
        return data

# 解析每页数据，获取商家信息
def get_shops(data, categoryName):
    onePageData = pd.DataFrame()
    for eachShopInfo in data.get('poiLists').get('poiInfos'):
        shopId = eachShopInfo.get('poiId')
        shopName = eachShopInfo.get('title')
        avgScore = eachShopInfo.get('avgScore')
        allCommentNum = eachShopInfo.get('allCommentNum')
        address = eachShopInfo.get('address')
        avgPrice = eachShopInfo.get('avgPrice')
        onePageData = onePageData.append(pd.DataFrame({'商家ID':[shopId],
                                                       '商家名称':[shopName],
                                                       '城市':[cityName],
                                                       '一级类目':[className],
                                                       '二级类目':[categoryName],
                                                       '评分':[avgScore],
                                                       '评论数':[allCommentNum],
                                                       '人均':[avgPrice],
                                                       '地址':[address]}))
    return onePageData

# 获取一个城市多区域多类目商家信息
def get_shops_category_area(city, df_category, df_area):
    for index, crow in df_category.iterrows():
        categoryName = crow['categoryName']
        categoryID = crow['categoryID']
        for index, arow in df_area.iterrows():
            areaName = arow['areaName']
            areaID = arow['areaID']
            url = 'https://' + city + '.meituan.com/meishi/' + categoryID + areaID + '/' #更改城市   
            print(url)

            data = get_url(url)
            shopCounts = data.get('poiLists').get('totalCounts')
            pageNum = math.ceil(shopCounts/15) #美团一页展示15个商家
            print('{0}{1}：共{2}页，{3}个商家信息'.format(categoryName, areaName, pageNum, shopCounts))

            totalData = pd.DataFrame()
            for i in range(1, pageNum+1):
                print('获取第%d页' % i)
                url_page = url + 'pn{0}/'.format(i)
                data_page = get_url(url_page)
                onePageData = get_shops(data_page, categoryName)
                totalData = totalData.append(onePageData)
            
            cwd = os.getcwd()
            if not os.path.exists(cwd + '/' + cityName):
                os.makedirs(cwd + '/' + cityName)
            if not os.path.exists(cwd + '/' + cityName + '/' + categoryName):
                os.mkdir(cwd + '/' + cityName + '/' + categoryName)
            totalData.to_csv(cwd + '/' + cityName + '/' + categoryName + '/' + areaName + str(totalData.shape[0]) + '.csv', index=False, encoding='utf_8_sig')     

In [ ]:
# 广州
city = 'gz'
cityName = '广州'
className = '美食'

df_category = pd.DataFrame ({'categoryName': ['蛋糕甜点','火锅','日韩料理','西餐','自助餐','海鲜',
                                              '烧烤烤肉','中式烧烤烤串','香锅烤鱼',
                                              '川湘菜','江浙菜','粤菜'],
                             'categoryID': ['c11','c17','c28','c35','c40','c63',
                                            'c54','c400','c20004',
                                            'c55','c56','c57']})
print(df_category)

df_area_gz = pd.DataFrame ({'areaName': ['越秀区','天河区','番禺区','海珠区','白云区',
                                         '荔湾区','黄埔区','增城市','花都区','从化市',
                                         '南沙区'],
                            'areaID': ['b23','b22','b274','b24','b26',
                                       'b25','b737','b739','b738','b740',
                                       'b1068']})
print(df_area_gz)

get_shops_category_area(city, df_category, df_area_gz)

In [ ]:
# 杭州
city = 'hz'
cityName = '杭州'
className = '美食'

df_category = pd.DataFrame ({'categoryName': ['蛋糕甜点','火锅','日韩料理','西餐','自助餐','海鲜',
                                              '烧烤烤肉','中式烧烤烤串','香锅烤鱼',
                                              '川湘菜','江浙菜','粤菜'],
                             'categoryID': ['c11','c17','c28','c35','c40','c63',
                                            'c54','c400','c20004',
                                            'c55','c56','c57']})
print(df_category)

df_area_hz = pd.DataFrame ({'areaName': ['西湖区','拱墅区','萧山区','江干区','滨江区',
                                         '余杭区','上城区','临安市','富阳区','下城区',
                                         '桐庐县','建德市','淳安县'],
                            'areaID': ['b59','b57','b5225','b58','b60',
                                       'b2779','b55','b2780','b2781','b56',
                                       'b2783','b2782','b2784']})
print(df_area_hz)

get_shops_category_area(city, df_category, df_area_hz)

In [ ]:
# 上海
city = 'sh'
cityName = '上海'
className = '美食'

df_category = pd.DataFrame ({'categoryName': ['蛋糕甜点','火锅','日本料理','韩国料理','西餐','自助餐','海鲜',
                                              '烧烤烤肉','中式烧烤烤串','香锅烤鱼',
                                              '川湘菜','江浙菜','粤菜'],
                             'categoryID': ['c11','c17','c20059','c20060','c35','c40','c63',
                                            'c54','c400','c20004',
                                            'c55','c56','c57']})
print(df_category)


df_area_sh = pd.DataFrame ({'areaName': ['浦东新区','徐汇区','长宁区','黄浦区','静安区',
                                         '闵行区','杨浦区','普陀区','虹口区','宝山区',
                                         '松江区','嘉定区','金山区','青浦区','奉贤区',
                                         '崇明区'],
                            'areaID': ['b5','b2','b4','b6','b3',
                                       'b12','b10','b7','b9','b13',
                                       'b4737','b4738','b665','b4739','b664',
                                       'b666']})
print(df_area_sh)

get_shops_category_area(city, df_category, df_area_sh)

In [ ]:
# 珠海
city = 'zh'
cityName = '珠海'
className = '美食'

df_category = pd.DataFrame ({'categoryName': ['蛋糕甜点','火锅','日韩料理','西餐','自助餐','海鲜',
                                              '烧烤烤肉','中式烧烤烤串','香锅烤鱼',
                                              '川湘菜','江浙菜','粤菜'],
                             'categoryID': ['c11','c17','c28','c35','c40','c63',
                                            'c54','c400','c20004',
                                            'c55','c56','c57']})
print(df_category)

df_area_zh = pd.DataFrame ({'areaName': ['香洲区','斗门区','金湾区','横琴新区'],
                            'areaID': ['b162','b163','b164','b7004']})
print(df_area_zh)

get_shops_category_area(city, df_category, df_area_zh)

In [ ]:
# 合肥
city = 'hf'
cityName = '合肥'
className = '美食'

df_category = pd.DataFrame ({'categoryName': ['蛋糕甜点','火锅','日韩料理','西餐','自助餐','海鲜',
                                              '烧烤烤肉','中式烧烤烤串','香锅烤鱼',
                                              '川湘菜','江浙菜','粤菜'],
                             'categoryID': ['c11','c17','c28','c35','c40','c63',
                                            'c54','c400','c20004',
                                            'c55','c56','c57']})
print(df_category)

df_area_hf = pd.DataFrame ({'areaName': ['蜀山区','庐阳区','瑶海区','包河区','经开区',
                                         '政务区','新站区','高新区','巢湖市','肥东县',
                                         '肥西县','长丰县','庐江县'],
                            'areaID': ['b199','b198','b197','b200','b5729',
                                       'b5733','b5731','b5732','b7607','b2853',
                                       'b2854','b2852','b7205']})
print(df_area_hf)

get_shops_category_area(city, df_category, df_area_hf)

In [ ]:
# 青岛
city = 'qd'
cityName = '青岛'
className = '美食'

df_category = pd.DataFrame ({'categoryName': ['蛋糕甜点','火锅','日韩料理','西餐','自助餐','海鲜',
                                              '烧烤烤肉','中式烧烤烤串','香锅烤鱼',
                                              '川湘菜','江浙菜','粤菜'],
                             'categoryID': ['c11','c17','c28','c35','c40','c63',
                                            'c54','c400','c20004',
                                            'c55','c56','c57']})
print(df_category)

df_area_qd = pd.DataFrame ({'areaName': ['市北区','市南区','李沧区','崂山区','黄岛区',
                                         '城阳区','即墨市','胶州市','平度市','莱西市'],
                            'areaID': ['b133','b132','b135','b136','b137', 
                                       'b3185','b3187','b3186','b3188','b3190']})
print(df_area_qd)

get_shops_category_area(city, df_category, df_area_qd)

In [ ]:
# 天津
city = 'tj'
cityName = '天津'
className = '美食'

df_category = pd.DataFrame ({'categoryName': ['蛋糕甜点','火锅','日韩料理','西餐','自助餐','海鲜',
                                              '烧烤烤肉','中式烧烤烤串','香锅烤鱼',
                                              '川湘菜','江浙菜','粤菜'],
                             'categoryID': ['c11','c17','c28','c35','c40','c63',
                                            'c54','c400','c20004',
                                            'c55','c56','c57']})
print(df_category)

df_area_tj = pd.DataFrame ({'areaName': ['滨海新区','南开区','和平区','红桥区','河西区',
                                         '河东区','河北区','北辰区','津南区','东丽区',
                                         '西青区','蓟州区','武清区','静海区','宝坻区',
                                         '宁河区'],
                            'areaID': ['b1693','b49','b48','b53','b50', 
                                       'b51','b52','b2079','b2078','b1171',
                                       'b2077','b2082','b2080','b2084','b2081', 
                                       'b2083']})
print(df_area_tj)

get_shops_category_area(city, df_category, df_area_tj)

In [ ]:
# 徐州
city = 'xz'
cityName = '徐州'
className = '美食'

df_category = pd.DataFrame ({'categoryName': ['蛋糕甜点','火锅','日韩料理','西餐','自助餐','海鲜',
                                              '烧烤烤肉','中式烧烤烤串','香锅烤鱼',
                                              '川湘菜','江浙菜','粤菜'],
                             'categoryID': ['c11','c17','c28','c35','c40','c63',
                                            'c54','c400','c20004',
                                            'c55','c56','c57']})
print(df_category)

df_area_xz = pd.DataFrame ({'areaName': ['鼓楼区','贾汪区','云龙区','泉山区','邳州市',
                                         '新沂市','睢宁县','沛县','丰县','铜山区'],
                            'areaID': ['b2711','b2713','b2710','b2714','b2715',
                                       'b2716','b2718','b2719','b2720','b5914']})
print(df_area_xz)

get_shops_category_area(city, df_category, df_area_xz)

In [ ]:
编码列表：
分类：
蛋糕甜点c11 火锅c17 日韩料理c28 西餐c35 自助餐c40 海鲜c63
川湘菜c55 江浙菜c56 粤菜c57
烧烤烤肉c54 中式烧烤烤串c400 香锅烤鱼c20004
（上海）日本料理c20059 韩国料理c20060

区域：
广州：
越秀区b23 天河区b22 番禺区b274 海珠区b24 白云区b26 荔湾区b25 黄埔区b737 增城市b739 花都区b738 从化市b740 南沙区b1068

杭州：
西湖区b59 拱墅区b57 萧山区b5225 江干区b58 滨江区b60 余杭区b2779 上城区b55 临安市b2780 富阳区b2781 下城区b56 桐庐县b2783 建德市b2782 淳安县b2784

上海：
浦东新区b5 徐汇区b2 长宁区b4 黄浦区b6 静安区b3 闵行区b12 杨浦区b10 普陀区b7 虹口区b9 宝山区b13 松江区b4737 嘉定区b4738 金山区b665 青浦区b4739 奉贤区b664 崇明区b666

珠海：
香洲区b162 斗门区b163 金湾区b164 横琴新区b7004 

合肥：
蜀山区b199 庐阳区b198 瑶海区b197 包河区b200 经开区b5729 政务区b5733 新站区b5731 高新区b5732 巢湖市b7607 肥东县b2853 肥西县b2854 长丰县b2852 庐江县b7205

青岛：
市北区b133 市南区b132 李沧区b135 崂山区b136 黄岛区b137 城阳区b3185 即墨市b3187 胶州市b3186 平度市b3188 莱西市b3190

天津：
滨海新区b1693 南开区b49 和平区b48 红桥区b53 河西区b50 河东区b51 河北区b52 北辰区b2079 津南区b2078 东丽区b1171 西青区b2077 蓟州区b2082 武清区b2080 静海区b2084 宝坻区b2081 宁河区b2083

徐州：
鼓楼区b2711 贾汪区b2713 云龙区b2710 泉山区b2714 邳州市b2715 新沂市b2716 睢宁县b2718 沛县b2719 丰县b2720 铜山区b5914 

In [3]:
# 获取一个城市单区域单类目商家信息
city = 'gz'
cityName = '广州'
className = '美食'
categoryName = '火锅'
categoryID = 'c17'
areaName = '越秀区'
areaID = 'b23' 
url = 'https://gz.meituan.com/meishi/' + categoryID + areaID + '/'   
print(url)

data = get_url(url)
shopCounts = data.get('poiLists').get('totalCounts')
pageNum = math.ceil(shopCounts/15) # 美团一页展示15个商家
print('共{0}页，{1}个商家信息'.format(pageNum, shopCounts))

totalData = pd.DataFrame()
for i in range(1, pageNum+1):
    url_page = url + 'pn{0}/'.format(i)
    data_page = get_url(url_page)
    print('获取第%d页' % i)
    onePageData = get_shops(data_page, categoryName)
    totalData = totalData.append(onePageData)

cwd = os.getcwd()
if not os.path.exists(cwd + '/' + cityName):
    os.makedirs(cwd + '/' + cityName)
if not os.path.exists(cwd + '/' + cityName + '/' + categoryName):
    os.mkdir(cwd + '/' + cityName + '/' + categoryName)
totalData.to_csv(cwd + '/' + cityName + '/' + categoryName + '/' + areaName + str(totalData.shape[0]) + '.csv', index=False, encoding='utf_8_sig')   

https://gz.meituan.com/meishi/c17b23/
共9页，129个商家信息
获取第1页
获取第2页
获取第3页
获取第4页
获取第5页
获取第6页
获取第7页
获取第8页
获取第9页
